In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:86% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 데이터베이스 내에 넣을 데이터프레임 가공 -> 저장

# 순서 - master_crop_variety -> map_region_weather_station -> weather_daily -> factor_external -> fact_trade

In [2]:
import pandas as pd
import os
import re
import urllib.parse
from dotenv import load_dotenv
from datetime import datetime, date
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

load_dotenv()

True

In [3]:
# 사전 DB 세팅 # 외부 세팅인 첨부한 .env 설정파일 참고해서 env 설정하자
# DB 정보
user = os.getenv("DB_USER")  # .env 파일에 DB_USER 설정해도 됨
host = os.getenv("DB_HOST")
password = os.getenv("DB_PW")  # DB_pw → 대소문자 주의 (env 키명)
password  = urllib.parse.quote_plus(password)
port = int(os.getenv("DB_PORT"))
db = os.getenv("DB_NAME")
engine = create_engine(
    f"mysql+pymysql://{user}:{password}@{host}:{port}/{db}?charset=utf8mb4"
)

# master_crop_variety(품목코드) DB 삽입

In [59]:
import pandas as pd
import numpy as np

# 데이터 로드
df = pd.read_csv('datasets/master_crop_variety.csv', encoding='utf8')

# 코드 조합 함수
def create_crop_codes(df: pd.DataFrame) -> pd.DataFrame:
    """
    주어진 DataFrame에 품목 및 품종 코드를 생성합니다.
    """
    # 필수 컬럼 존재 여부 확인
    required_cols = ['gds_lclsf_cd', 'gds_mclsf_cd', 'gds_sclsf_cd']
    if not all(col in df.columns for col in required_cols):
        print(f"오류: 필수 컬럼이 DataFrame에 없습니다: {required_cols}")
        return df

    # 컬럼을 문자열로 변환하고 2자리로 패딩
    for col in required_cols:
        df[col] = df[col].astype(str).str.zfill(2)

    # 'item_code' 생성: 대분류 + 품목 코드
    df['item_code'] = df['gds_lclsf_cd'] + df['gds_mclsf_cd']

    # 'crop_full_code' 생성: 대분류 + 품목 + 품종 코드
    df['crop_full_code'] = df['gds_lclsf_cd'] + df['gds_mclsf_cd'] + df['gds_sclsf_cd']

    return df

df = create_crop_codes(df)

In [61]:
# 백업
df.drop_duplicates(inplace=True)
df.to_csv('datasets//master_crop_variety_2.csv', encoding='utf-8')

In [12]:
df = pd.read_csv('datasets/master_crop_variety_2.csv', encoding='utf8')

In [65]:
# DB로 저장
# to_sql로 insert (테이블명, 커넥션, 옵션)
df.to_sql(
    name='master_crop_variety',    # 실제 DB의 테이블명
    con=engine,
    if_exists='append',            # append: 추가 / replace: 전체 덮어쓰기
    index=False
)

print("DB 적재 완료!")

DB 적재 완료!


# 산지코드-직팜코드-관측소코드 매핑

In [15]:
df = pd.read_csv('datasets/산지코드_직팜_관측지점_매핑완료_수정.csv', encoding='cp949')

In [16]:
def expand_row(row):
    if '~' in str(row['산지코드']):
        start, end = map(int, row['산지코드'].split('~'))
        return [
            {**row, '산지코드': code}
            for code in range(start, end+1)
        ]
    else:
        return [{**row, '산지코드': int(row['산지코드'])}]

# 예시 DataFrame: df
expanded = []
for _, row in df.iterrows():
    expanded.extend(expand_row(row))

df_expanded = pd.DataFrame(expanded)
df = df_expanded.rename(columns={'산지코드' : 'plor_cd',
                            '산지이름': 'plor_nm',
                            '직팜산지코드': 'j_sanji_cd', 
                            '직팜산지이름': 'j_sanji_nm', 
                            '관측지점' : 'station_cd'
                           })
df

,plor_cd,plor_nm,j_sanji_cd,j_sanji_nm,위도,경도,station_cd
0,100000,서울특별시,1000,서울특별시,37.5641,126.9970,108.0
1,100001,서울특별시,1000,서울특별시,37.5641,126.9970,108.0
2,100002,서울특별시,1000,서울특별시,37.5641,126.9970,108.0
3,100003,서울특별시,1000,서울특별시,37.5641,126.9970,108.0
4,100004,서울특별시,1000,서울특별시,37.5641,126.9970,108.0
...,...,...,...,...,...,...,...
800030,971000,경상남도 창원시,1120,경상남도 창원시,35.2372,128.6811,255.0
800031,980000,경상북도,1169,경상북도,36.5681,128.7293,136.0
800032,981000,경상북도 포항시,1159,경상북도 포항시,36.0194,129.3434,138.0
800033,990000,제주도,1170,제주특별자치도,33.4996,126.5312,184.0


In [19]:
# 불필요할 칼럼/결측행 제거
df.drop(columns=['위도', '경도'], inplace=True)
df.dropna(inplace=True)

# station_cd 속성 변경
df['station_cd'] = df['station_cd'].astype(int)

# 백업
df.to_csv('datasets/map_region_weather_station_utf-8.csv', encoding='utf-8', index=False)

In [66]:
df = pd.read_csv('datasets/map_region_weather_station_utf-8.csv', encoding='utf-8')

In [67]:
# 데이터베이스 저장
# to_sql로 insert (테이블명, 커넥션, 옵션)
df.to_sql(
    name='map_region_weather_station',    # 실제 DB의 테이블명
    con=engine,
    if_exists='append',            # append: 추가 / replace: 전체 덮어쓰기
    index=False
)

print("DB 적재 완료!")

DB 적재 완료!


# 일별 기상 데이터 

In [79]:
df = pd.read_csv('datasets/기상청_서울_일기요소_20180101-20250531.csv', encoding='cp949')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257907 entries, 0 to 257906
Data columns (total 56 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   TM             257907 non-null  int64  
 1   STN            257907 non-null  int64  
 2   WS_AVG         257907 non-null  float64
 3   WR_DAY         257907 non-null  int64  
 4   WD_MAX         257907 non-null  int64  
 5   WS_MAX         257907 non-null  float64
 6   WS_MAX_TM      257907 non-null  int64  
 7   WD_INS         257907 non-null  int64  
 8   WS_INS         257907 non-null  float64
 9   WS_INS_TM      257907 non-null  int64  
 10  TA_AVG         257907 non-null  float64
 11  TA_MAX         257907 non-null  float64
 12  TA_MAX_TM      257907 non-null  int64  
 13  TA_MIN         257907 non-null  float64
 14  TA_MIN_TM      257907 non-null  int64  
 15  TD_AVG         257907 non-null  float64
 16  TS_AVG         257907 non-null  float64
 17  TG_MIN         257907 non-nul

In [80]:
df= df.loc[:, ['TM', 'STN', 'TA_AVG', 'TA_MAX', 'TA_MIN', 'HM_AVG', 'RN_DAY', 'RN_60M_MAX']]
# 강수량, 1시간최고 강수량은 결측치(-9) 혹은 비가 안옴(0)이 많아 0 이하는 0으로 처리
# merged_df['강수량(mm)'] = merged_df['강수량(mm)'<=0].count()
df.loc[df['RN_DAY']<=0, 'RN_DAY'] = 0
df.loc[df['RN_60M_MAX']<=0, 'RN_60M_MAX'] = 0
df['TM'] = pd.to_datetime(df['TM'].astype(str), format='%Y%m%d')
df.to_csv('datasets/weather_daily.csv', encoding='utf-8', index=False)

In [68]:
df = pd.read_csv('datasets/weather_daily.csv', encoding='utf-8')

In [69]:
#데이터베이스 저장
# to_sql로 insert (테이블명, 커넥션, 옵션)
df.to_sql(
    name='weather_daily',    # 실제 DB의 테이블명
    con=engine,
    if_exists='append',            # append: 추가 / replace: 전체 덮어쓰기
    index=False
)

print("DB 적재 완료!")

DB 적재 완료!


# 외생변수 테이블

In [30]:
import pandas as pd
# 외생요소 가공
df_factor_external = pd.read_csv('datasets/holiday_작기.csv', encoding='utf-8')

# 멜팅
df_melted = df_factor_external.melt(
    id_vars=['date', 'holiday_flag', 'holiday_score'],  # 고정 컬럼
    var_name='crop_name',         # 새로 생길 품목명 컬럼명
    value_name='grow_score'       # 각 품목의 작기지수 값 컬럼명
)

# 중복 제거
df_melted = df_melted.drop_duplicates()

# 4인덱스 리셋
df = df_melted.reset_index(drop=True)

         date  holiday_flag  holiday_score crop_name  grow_score
0  2018-01-01             1            0.0        배추         1.0
1  2018-01-02             0            0.0        배추         1.0
2  2018-01-03             0            0.0        배추         1.0
3  2018-01-04             0            0.0        배추         1.0
4  2018-01-05             0            0.0        배추         1.0


In [31]:
# 품목
# 1. 품목명 → 품목코드 매핑 딕셔너리 수동 작성
crop_code_map = {
    "양파": "1201",  
    "배추": "1001", 
    "상추": "1005",  
    "과수": "0601",   
    "무" : "1101",
#    '배' : '0602',
    "마늘" : "1209",
    "건고추" : "1207",
    "감자" : "0501",
}

# 2. crop_name → crop_code로 변환 컬럼 추가
df['item_code'] = df['crop_name'].map(crop_code_map)
df.drop(colums='crop_name', inplace=True)

# 3. 중간 저장 (데일리) - 혹시나 쓰거나 기준이 바뀔수 있으니 백업
df.to_csv('datasets/factor_external_daily.csv', encoding='utf-8', index=False)

In [32]:
# 주간 병합하기
# 주차 식별 칼럼 만들기
def get_week_of_year(date):
    date = pd.to_datetime(date)
    year = date.year
    week_number = date.isocalendar().week
    return f"{year}{week_number:02d}"

df['week_no'] = df['date'].apply(get_week_of_year)

# 중복 제거
df.drop_duplicates(inplace=True)
df.drop(columns='date', inplace=True)

# 주차와 코드로 병합하기
df = df.groupby(['week_no', 'item_code']).sum(['holiday_flag', 'holiday_score', 'grow_score']).reset_index().sort_values(by='week_no', ascending=True)
df.to_csv('datasets/factor_external_weekly.csv', encoding='utf-8', index=False)

In [33]:
# 데이터 베이스 저장
# to_sql로 insert (테이블명, 커넥션, 옵션)
df.to_sql(
    name='factor_external_weekly',    # 실제 DB의 테이블명
    con=engine,
    if_exists='append',            # append: 추가 / replace: 전체 덮어쓰기
    index=False
)

print("DB 적재 완료!")

DB 적재 완료!


In [3]:
# 추가! 배
# 작기 정보 로드
df_grow = pd.read_csv('datasets/factor_external_weekly.csv', encoding='utf-8')
df_grow2 = df_grow[df_grow['item_code']==601]
df_grow2.to_sql(
    name='factor_external_weekly',    # 실제 DB의 테이블명
    con=engine,
    if_exists='append',            # append: 추가 / replace: 전체 덮어쓰기
    index=False
)

print("DB 적재 완료!")

DB 적재 완료!


# 거래데이터 삽입_daily

In [24]:
# 원본에서 출발
df1 = pd.read_csv('C:/AI-X/source/00_project/data/사과/유통공사_사과_retry_성공_20250718_083528.csv', encoding='cp949')
df2 = pd.read_csv('C:/AI-X/source/00_project/data/사과/유통공사_도매시장_사과_20180101-20250531.csv', encoding='cp949')
#df3 = pd.read_csv('datasets/legacy/유통공사_retry_양파_2(완).csv', encoding='cp949')
df = pd.concat([df1, df2], axis=0)
df.drop_duplicates(inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_23680\1757125204.py:3: DtypeWarning: Columns (7,9,13,15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('C:/AI-X/source/00_project/data/사과/유통공사_도매시장_사과_20180101-20250531.csv', encoding='cp949')


In [25]:
# plor_cd가 문자열이 아닐 가능성 대비
df['plor_cd'] = df['plor_cd'].fillna('').astype(str)

pattern = r'^[^0-9]+$'

condition = (
    df['totprc'].isna() | (df['totprc'] <= 0) |
    df['unit_tot_qty'].isna() | (df['unit_tot_qty'] <= 0) |
    df['plor_cd'].str.strip().isin(['0', '0.0']) |
    df['plor_cd'].str.match(pattern, na=False) |
    df['plor_nm'].isna() |
    (df['plor_nm'] == 0)
)

# 조건에 해당하는 행 추출
df_filtered = df[~condition]

# 수입산을 하나로 몰까 했지만.. 그냥 패스
# df.loc[df['plor_cd'].str.startswith('800')]['plor_cd'] = '800000'

# 직팜코드 테이블 소환 - 향후 이걸 DB로 가져오자
df_region = pd.read_csv('datasets/map_region_weather_station_utf-8.csv', encoding='utf-8')

# 직팜코드 붙이기
df_region['plor_cd'] = df_region['plor_cd'].astype(str)
df_merged_1 = pd.merge(df_filtered, df_region[['plor_cd', 'j_sanji_cd']],  on='plor_cd', how='left')
df_merged_1['trd_clcln_ymd'] = pd.to_datetime(df_merged_1['trd_clcln_ymd'], format='%Y-%m-%d')

# 아이템 풀코드 장착
for col in ['gds_lclsf_cd', 'gds_mclsf_cd', 'gds_sclsf_cd']:
    df_merged_1[col] = df_merged_1[col].astype(str) 
    df_merged_1[col] = df_merged_1[col].str.zfill(2)
df_merged_1['crop_full_code'] = df_merged_1['gds_lclsf_cd']+df_merged_1['gds_mclsf_cd']+df_merged_1['gds_sclsf_cd']

# 필요한 열만 선별

df = df_merged_1[['trd_clcln_ymd', 'crop_full_code', 'j_sanji_cd', 'unit_tot_qty', 'totprc']]
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4207547 entries, 0 to 4207546
Data columns (total 5 columns):
 #   Column          Dtype         
---  ------          -----         
 0   trd_clcln_ymd   datetime64[ns]
 1   crop_full_code  object        
 2   j_sanji_cd      float64       
 3   unit_tot_qty    float64       
 4   totprc          float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 160.5+ MB


,trd_clcln_ymd,crop_full_code,j_sanji_cd,unit_tot_qty,totprc
0,2018-04-24,060103,1079.0,50.0,140000.0
1,2018-04-24,060103,1113.0,520.0,445000.0
2,2018-04-24,060103,1132.0,540.0,1086500.0
3,2018-04-24,060103,1144.0,610.0,1231000.0
4,2018-04-24,060103,1151.0,1642.0,2463000.0


In [74]:
# 데이터 머지 (일, 작물코드, 산지)

df_merged = df.groupby(['trd_clcln_ymd', 'crop_full_code', 'j_sanji_cd']).sum(['unit_tot_qty', 'totprc']).reset_index()
df_merged['avg_prc'] = round(df_merged['totprc'] / df_merged['unit_tot_qty'])
df_merged

,trd_clcln_ymd,crop_full_code,j_sanji_cd,unit_tot_qty,totprc,avg_prc
0,2018-01-02,060100,1047.0,4194.0,6433800.0,1534.0
1,2018-01-02,060101,1154.0,800.0,2097600.0,2622.0
2,2018-01-02,060103,1008.0,816.0,2005200.0,2457.0
3,2018-01-02,060103,1022.0,1760.0,3499600.0,1988.0
4,2018-01-02,060103,1036.0,3450.0,7784100.0,2256.0
...,...,...,...,...,...,...
284124,2025-05-30,060199,1148.0,3456.0,20664000.0,5979.0
284125,2025-05-30,060199,1151.0,1760.0,3960000.0,2250.0
284126,2025-05-30,060199,1156.0,1660.0,7401800.0,4459.0
284127,2025-05-30,0601A2,1159.0,230.0,1390800.0,6047.0


In [75]:
# 등급 라벨링

# j_sanji_cd <= 2000 인 값만 필터링
mask_domestic = df_merged['j_sanji_cd'] < 2000

# trd_clcln_ymd 기준으로 그룹화하여 각 그룹별 avg_prc의 80%, 20% 분위 계산
quantiles = df_merged[mask_domestic].groupby('trd_clcln_ymd')['avg_prc'].quantile([0.2, 0.8]).unstack()

# 함수 정의: trd_clcln_ymd와 avg_prc 기준으로 '고', '중', '저' 구분
def assign_grade(row):
    if row['j_sanji_cd'] > 2000:
        return '수입'
    q20 = quantiles.loc[row['trd_clcln_ymd'], 0.2]
    q80 = quantiles.loc[row['trd_clcln_ymd'], 0.8]
    if row['avg_prc'] >= q80:
        return '고'
    elif row['avg_prc'] >= q20:
        return '중'
    else:
        return '저'

# grade_label 컬럼 생성
df_merged['grade_label'] = df_merged.apply(assign_grade, axis=1)
df_merged

,trd_clcln_ymd,crop_full_code,j_sanji_cd,unit_tot_qty,totprc,avg_prc,grade_label
0,2018-01-02,060100,1047.0,4194.0,6433800.0,1534.0,저
1,2018-01-02,060101,1154.0,800.0,2097600.0,2622.0,중
2,2018-01-02,060103,1008.0,816.0,2005200.0,2457.0,중
3,2018-01-02,060103,1022.0,1760.0,3499600.0,1988.0,중
4,2018-01-02,060103,1036.0,3450.0,7784100.0,2256.0,중
...,...,...,...,...,...,...,...
284124,2025-05-30,060199,1148.0,3456.0,20664000.0,5979.0,중
284125,2025-05-30,060199,1151.0,1760.0,3960000.0,2250.0,저
284126,2025-05-30,060199,1156.0,1660.0,7401800.0,4459.0,중
284127,2025-05-30,0601A2,1159.0,230.0,1390800.0,6047.0,고


In [76]:
df_merged.drop(columns='avg_prc', inplace=True)

In [77]:
df_merged.to_csv('datasets/fact_trade_사과.csv', encoding='utf-8', index=False)

In [70]:
df_merged = pd.read_csv('datasets/fact_trade.csv', encoding='utf-8')

In [50]:
# 데이터 베이스 저장
# to_sql로 insert (테이블명, 커넥션, 옵션)
df_merged.to_sql(
    name='fact_trade',    # 실제 DB의 테이블명
    con=engine,
    if_exists='append',            # append: 추가 / replace: 전체 덮어쓰기
    index=False
)

print("DB 적재 완료!")

DB 적재 완료!


# 거래데이터 삽입_weekly

In [43]:
# 사전 DB 세팅된 것이 있어야 가능!
ITEM = "무"  # 맨 아래 백업 파일명에 들어갈 작물명!
query = """
SELECT *
FROM fact_trade
WHERE crop_full_code NOT LIKE '0601%%'
    AND crop_full_code NOT LIKE '1101%%'
    AND crop_full_code NOT LIKE '1201%%'
"""
# 만일 작물을 하나씩 추가하려고 한다면 위와 같이 아이템코드를 넣기
df_trade = pd.read_sql(query, engine)
df_trade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 826656 entries, 0 to 826655
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   trd_clcln_ymd   826656 non-null  object 
 1   crop_full_code  826656 non-null  object 
 2   item_code       0 non-null       object 
 3   j_sanji_cd      826656 non-null  object 
 4   grade_label     826656 non-null  object 
 5   unit_tot_qty    826656 non-null  float64
 6   totprc          826656 non-null  float64
 7   year_part       826656 non-null  int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 50.5+ MB


In [46]:
df_trade['item_code'] = df_trade['crop_full_code'].str[:4]

In [49]:
df_trade.drop(columns='year_part', inplace=True)

In [42]:
# 사전 DB 세팅된 것이 있어야 가능!
ITEM = "무"  # 맨 아래 백업 파일명에 들어갈 작물명!
query = """
SELECT *
FROM fact_trade
WHERE crop_full_code NOT LIKE '0601%%'

    AND crop_full_code NOT LIKE '1201%%'
"""
# 만일 작물을 하나씩 추가하려고 한다면 위와 같이 아이템코드를 넣기
df_trade = pd.read_sql(query, engine)
df_trade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170765 entries, 0 to 170764
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   trd_clcln_ymd   170765 non-null  object 
 1   crop_full_code  170765 non-null  object 
 2   item_code       170765 non-null  object 
 3   j_sanji_cd      170765 non-null  object 
 4   grade_label     170765 non-null  object 
 5   unit_tot_qty    170765 non-null  float64
 6   totprc          170765 non-null  float64
 7   year_part       170765 non-null  int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 10.4+ MB


In [24]:
%%time
# 1작물 할때 20초 정도 걸림! 참고!
##주간 병합하기 
# 주차 표기
def get_week_of_year(date):
    date = pd.to_datetime(date)
    year = date.year
    week_number = date.isocalendar().week
    return f"{year}{week_number:02d}"

df_trade['weekno'] = df_trade['trd_clcln_ymd'].apply(get_week_of_year)

# 데이터 머지 (일, 작물코드, 산지)
df_trade.drop(columns='grade_label', inplace=True)
df_merged = df_trade.groupby(["weekno", "crop_full_code", "item_code", "j_sanji_cd", "year_part"]).sum(['unit_tot_qty','totprc']).reset_index()

# 주간 평균 삽입
df_merged['avg_prc'] = round(df_merged['totprc'] / df_merged['unit_tot_qty'])

## 등급 라벨링

# j_sanji_cd != 2000 인 값만 필터링
mask_domestic = df_merged['j_sanji_cd'] != '2000'

# trd_clcln_ymd 기준으로 그룹화하여 각 그룹별 avg_prc의 80%, 20% 분위 계산
quantiles = df_merged[mask_domestic].groupby('weekno')['avg_prc'].quantile([0.2, 0.8]).unstack()

# 함수 정의: trd_clcln_ymd와 avg_prc 기준으로 '고', '중', '저' 구분
def assign_grade(row):
    if row['j_sanji_cd'] == '2000':
        return '수입'
    q20 = quantiles.loc[row['weekno'], 0.2]
    q80 = quantiles.loc[row['weekno'], 0.8]
    if row['avg_prc'] >= q80:
        return '고'
    elif row['avg_prc'] >= q20:
        return '중'
    else:
        return '저'

# grade_label 컬럼 생성
df_merged['grade_label'] = df_merged.apply(assign_grade, axis=1)
df_merged.drop_duplicates()
df_merged.drop(columns='year_part')

# 체크 및 필터링 (결측치`)
num_cols = ['totprc', 'unit_tot_qty', 'avg_prc']
for col in num_cols:
    df_merged[col] = pd.to_numeric(df_merged[col], errors='coerce')
    
df_merged.to_csv(f'datasets/fact_trade_weekly_{ITEM}_BACKUP.csv', encoding='cp949', index=False)

df_merged.info()
df_merged.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182237 entries, 0 to 182236
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   weekno          182237 non-null  object 
 1   crop_full_code  182237 non-null  object 
 2   item_code       182237 non-null  object 
 3   j_sanji_cd      182237 non-null  object 
 4   year_part       182237 non-null  int64  
 5   unit_tot_qty    182237 non-null  float64
 6   totprc          182237 non-null  float64
 7   avg_prc         182237 non-null  float64
 8   grade_label     182237 non-null  object 
dtypes: float64(3), int64(1), object(5)
memory usage: 12.5+ MB
CPU times: total: 21.5 s
Wall time: 21.7 s


,weekno,crop_full_code,item_code,j_sanji_cd,year_part,unit_tot_qty,totprc,avg_prc,grade_label
0,201801,060100,0601,1047,2018,4194.0,6433800.0,1534.0,중
1,201801,060100,0601,1106,2018,4300.0,6971800.0,1621.0,중
2,201801,060100,0601,1138,2018,78.0,312000.0,4000.0,고
3,201801,060101,0601,1144,2018,1500.0,6000000.0,4000.0,고
4,201801,060101,0601,1149,2018,3730.0,18650000.0,5000.0,고


In [36]:
# 데이터 베이스 저장
# to_sql로 insert (테이블명, 커넥션, 옵션)
df_merged.to_sql(
    name='fact_trade_weekly',    # 실제 DB의 테이블명
    con=engine,
    if_exists='append',            # append: 추가 / replace: 전체 덮어쓰기
    index=False
)

print("DB 적재 완료!")

DB 적재 완료!


In [51]:
# 사전 DB 세팅된 것이 있어야 가능!

query = """
SELECT *
FROM fact_trade_weekly
"""
# 만일 작물을 하나씩 추가하려고 한다면 위와 같이 아이템코드를 넣기
df_trade = pd.read_sql(query, engine)
df_trade.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 409888 entries, 0 to 409887
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   weekno          409888 non-null  object 
 1   crop_full_code  409888 non-null  object 
 2   item_code       409888 non-null  object 
 3   j_sanji_cd      409888 non-null  object 
 4   grade_label     409888 non-null  object 
 5   unit_tot_qty    409888 non-null  float64
 6   avg_prc         409888 non-null  float64
 7   year_part       409888 non-null  int64  
 8   totprc          409888 non-null  float64
dtypes: float64(3), int64(1), object(5)
memory usage: 28.1+ MB


In [52]:
df_trade['item_code'].value_counts()

item_code
1005    158262
0601    104189
1201     78048
1001     69389
Name: count, dtype: int64

# 예측데이터 삽입

In [140]:
import pandas as pd
from datetime import  timedelta
# 양파 : 1201 # 배추 : 1001 # 상추 : 1005 # 사과 : 0601 # 무 : 1101 # 감자 : 0501 # 대파 : 1202 # 건고추 : 1207
# 마늘 : 1209 # 딸기 : 0804  # 방울토마토 : 0806 # 오이 : 0901 # 양배추 : 1004  # 고구마 : 0502  # 배 : 0602

item='배추'
item_code = '1001'
OUTPUT_DIR = '../04_Modeling/forecast_results/'
file_path = f'{item}_price_forecast.csv'
save_path = f'datasets/{item}_price_forecast_정리.csv'
mid_save_path = f'../04_Modeling/datasets/작물_lag_월단위_스케일링X_인코딩X/{item}_월차_모델직전.csv' 
df_pred = pd.read_csv(OUTPUT_DIR+file_path)
df = pd.read_csv(mid_save_path)

## 예측테이블 수정
# 주차로 변경 + date 칼럼 삭제
df_pred['date'] = pd.to_datetime(df_pred['date'])
df_pred['year'] = df_pred['date'].dt.isocalendar().year
df_pred['week'] = df_pred['date'].dt.isocalendar().week
df_pred['week'] = df_pred['week'] -1
df_pred.drop(columns='date', inplace=True)
df_pred = df_pred.rename(columns={'predicted_price' : 'predict_avg_prc'})

# 현재 데이터 준비
df_curr = df.iloc[-10:,[0,1,5]]
curr_year = df_curr['year'].max()
curr_week_e = df_curr['week'].max()
curr_weekno = str(curr_year)+str(curr_week_e)
last_year = curr_year-1
last_week_s = curr_week_e - 10
last_week_e = curr_week_e + 4
df_curr = df_curr.rename(columns={'평균단가(원)' : 'current_avg_prc'})

# 전년도 데이터 준비
df_last = df[df['year']==last_year]
df_last = df_last.iloc[last_week_s:last_week_e,[0,1,5]]
df_last['year'] = curr_year
df_last = df_last.rename(columns={'평균단가(원)' : 'last_year_avg_prc'})

# 머지하기
df_merge1 = pd.merge(df_last, df_curr, 
                    on=['year', 'week'], how='left')
df_merge = pd.merge(df_merge1, df_pred, 
                    on=['year', 'week'], how='left')

# 아이템 정리 및 칼럼 정리
df_merge['item_code'] = item_code
df_merge['weekno'] = df_merge['year'].astype(str)+df_merge['week'].astype(str).str.zfill(2)
df_merge.drop(columns=['year', 'week'], inplace=True)

# 예측 그래프를 현재값과 이어 붙이기 위함
bridge_value = df_merge.loc[df_merge['weekno']==curr_weekno, 'current_avg_prc']
df_merge.loc[df_merge['weekno']==curr_weekno, 'predict_avg_prc'] = bridge_value

# 백업
df_merge.to_csv(save_path, index=False)
print('작업 및 백업 완료!')

작업 및 백업 완료!


In [141]:
# 데이터 베이스 저장
# to_sql로 insert (테이블명, 커넥션, 옵션)
df_merge.to_sql(
    name='predict_price',    # 실제 DB의 테이블명
    con=engine,
    if_exists='append',            # append: 추가 / replace: 전체 덮어쓰기
    index=False
)

print("DB 적재 완료!")

DB 적재 완료!


In [142]:
query = """
SELECT *
FROM predict_price
"""

df = pd.read_sql(query, engine)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42 entries, 0 to 41
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   weekno             42 non-null     object 
 1   item_code          42 non-null     object 
 2   current_avg_prc    30 non-null     float64
 3   last_year_avg_prc  42 non-null     float64
 4   predict_avg_prc    15 non-null     float64
dtypes: float64(3), object(2)
memory usage: 1.8+ KB


In [145]:
df.sort_values(by=['item_code', 'weekno'], ascending=True)

,weekno,item_code,current_avg_prc,last_year_avg_prc,predict_avg_prc
0,202513,1001,1377.840,1476.520,NaN
3,202514,1001,1378.350,1518.310,NaN
6,202515,1001,1343.780,1671.030,NaN
9,202516,1001,1200.780,1571.180,NaN
12,202517,1001,1028.900,1340.780,NaN
15,202518,1001,882.833,1371.640,NaN
18,202519,1001,769.672,1419.610,NaN
21,202520,1001,652.744,1032.460,NaN
24,202521,1001,591.382,669.479,NaN
27,202522,1001,627.872,707.135,627.872
